In [1]:
get_ipython().system('pip install transformers datasets scikit-learn pandas torch')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [2]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding, pipeline


In [30]:
df = pd.read_csv("amazon_reviews_full.csv")
df.tail()

,review_text,output
1495,Disappointing backpack. broke quickly and poor...,Negative
1496,Really disappointed with this smartphone. It’s...,Negative
1497,I regret buying this headphones. flimsy materi...,Negative
1498,This shirt is terrible. The performance malfun...,Negative
1499,This camera is terrible. The style broke quick...,Negative


In [ ]:
# df = df[['text', 'airline_sentiment']].dropna()
# df = df[df['airline_sentiment'].isin(['positive', 'neutral', 'negative'])]
# df = df.sample(3000, random_state=42).reset_index(drop=True)

In [5]:
# df['airline_sentiment'].value_counts()
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# df = (
#     df[df['airline_sentiment'].isin(['positive', 'neutral', 'negative'])]
#     .groupby('airline_sentiment', group_keys=False)
#     .apply(lambda x: x.sample(2000, random_state=42))
#     .reset_index(drop=True)
# )


In [31]:
df['output'].value_counts()

,count
output,
Positive,500
Neutral,500
Negative,500


In [32]:
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['output'])

In [33]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['review_text'].tolist(), df['label'].tolist(), test_size=0.2, random_state=42
)


In [35]:
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)


In [36]:
train_dataset = Dataset.from_dict({**train_encodings, 'label': train_labels})
val_dataset = Dataset.from_dict({**val_encodings, 'label': val_labels})


In [37]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = np.argmax(predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [39]:
from transformers import TrainingArguments, Trainer

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",              # where to save the model
    eval_strategy="epoch",         # evaluate at the end of each epoch
    save_strategy="epoch",               # save at the end of each epoch
    learning_rate=2e-5,                   # standard for fine-tuning
    per_device_train_batch_size=8,        # adjust based on GPU/CPU memory
    per_device_eval_batch_size=8,
    num_train_epochs=3,                   # you can change this
    weight_decay=0.01,                    # regularization
    logging_dir="./logs",                 # for TensorBoard
    logging_steps=50,
    load_best_model_at_end=True,          # load best model after training
    metric_for_best_model="accuracy",     # depends on compute_metrics
    greater_is_better=True,
    report_to="none"                     # if higher metric is better
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics

)



In [40]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics

)


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [41]:
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.012400,0.007099,1.000000,1.000000,1.000000,1.000000
2,0.004200,0.002960,1.000000,1.000000,1.000000,1.000000
3,0.003100,0.002348,1.000000,1.000000,1.000000,1.000000


TrainOutput(global_step=450, training_loss=0.09257908622423808, metrics={'train_runtime': 47.508, 'train_samples_per_second': 75.777, 'train_steps_per_second': 9.472, 'total_flos': 21422844165600.0, 'train_loss': 0.09257908622423808, 'epoch': 3.0})

In [42]:
eval_results = trainer.evaluate()

# Show evaluation metrics
print("📊 Evaluation Results:")
for key, value in eval_results.items():
    print(f"{key}: {value:.4f}")


📊 Evaluation Results:
eval_loss: 0.0071
eval_accuracy: 1.0000
eval_f1: 1.0000
eval_precision: 1.0000
eval_recall: 1.0000
eval_runtime: 0.3568
eval_samples_per_second: 840.7290
eval_steps_per_second: 106.4920
epoch: 3.0000


In [43]:
model.save_pretrained("amazon-model")
tokenizer.save_pretrained("amazon-model")


('amazon-model/tokenizer_config.json',
 'amazon-model/special_tokens_map.json',
 'amazon-model/vocab.txt',
 'amazon-model/added_tokens.json',
 'amazon-model/tokenizer.json')

In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

X = df['review_text']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

tfidf = TfidfVectorizer(max_features=3000)
X_train_vec = tfidf.fit_transform(X_train)
X_test_vec = tfidf.transform(X_test)

clf = LogisticRegression()
clf.fit(X_train_vec, y_train)

y_pred = clf.predict(X_test_vec)
print(classification_report(y_test, y_pred))



              precision    recall  f1-score   support

           0       1.00      1.00      1.00       110
           1       1.00      1.00      1.00        86
           2       1.00      1.00      1.00       104

    accuracy                           1.00       300
   macro avg       1.00      1.00      1.00       300
weighted avg       1.00      1.00      1.00       300



In [48]:
 from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

# Load model and tokenizer manually
model_path = "amazon-model"

model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# ✅ Fix label mapping (change these according to your training labels)
model.config.id2label = {0: "negative", 1: "neutral", 2: "positive"}
model.config.label2id = {"negative": 0, "neutral": 1, "positive": 2}

# Create pipeline with fixed label mapping
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

# Predict function
def predict_sentiment(text):
    result = classifier(text, truncation=True)[0]  # truncation to avoid length errors
    label = result['label']
    score = result['score']
    print(f"✈ Sentiment: {label} (Confidence: {score:.2f})")

# Test
test_tweet = input("Enter a tweet to analyze sentiment: ")
predict_sentiment(test_tweet)


Device set to use cuda:0


Enter a tweet to analyze sentiment: poor quality but delivery came early
✈ Sentiment: negative (Confidence: 0.98)


In [49]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

# Load a high-quality pretrained sentiment model
model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"

model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Create pipeline
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# Predict sentiment
def predict_sentiment(text):
    result = classifier(text, truncation=True)[0]
    label = result['label']
    score = result['score']
    print(f"✈ Sentiment: {label} (Confidence: {score:.2f})")

# Test
test_tweet = input("Enter a tweet to analyze sentiment: ")
predict_sentiment(test_tweet)


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


vocab.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cuda:0


Enter a tweet to analyze sentiment: poor quality and damaged one


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


✈ Sentiment: negative (Confidence: 0.87)


In [52]:
from google.colab import files
!zip -r amazon-model.zip amazon-model
files.download("amazon-model.zip")


  adding: amazon-model/ (stored 0%)
  adding: amazon-model/config.json (deflated 48%)
  adding: amazon-model/tokenizer.json (deflated 71%)
  adding: amazon-model/model.safetensors (deflated 8%)
  adding: amazon-model/vocab.txt (deflated 53%)
  adding: amazon-model/tokenizer_config.json (deflated 75%)
  adding: amazon-model/special_tokens_map.json (deflated 42%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>